In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from AVN import *
import scipy.special as scp
import os

In [ ]:
###Define colors code for plots:
from matplotlib import cm

##Get colors : 
cmap_name1 = cm.get_cmap('tab20', 20)
cmap_name2 = cm.get_cmap('tab20b', 20)
cmap_name3 = cm.get_cmap('tab20c', 20)
cmap_name4 = cm.get_cmap('tab10', 10)
cmap_name5 = cm.get_cmap('Set3', 12)

cmap1 = cmap_name1(np.linspace(0, 1, 20))
cmap2 = cmap_name2(np.linspace(0, 1, 20))
cmap3 = cmap_name3(np.linspace(0, 1, 20))
cmap4 = cmap_name4(np.linspace(0, 1, 10))
cmap5 = cmap_name5(np.linspace(0, 1, 12))

cmap = np.vstack([cmap1, cmap2, cmap3, cmap4, cmap5])

print(len(cmap))

In [ ]:
##Add migration swarms:
dfM = pd.read_csv("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas_23052024_2/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/dfM.csv")

##And without:
dfNM = pd.read_csv("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas_23052024_2/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/dfNM.csv")

dfM

In [ ]:
path_to_df0 = "/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas_23052024_2/new_all_run_files_for_etas/catalog_all/"

####Raw data:
df0 = pd.read_csv(path_to_df0+'raw_data_modified_version.csv')
###Structure : t, x, y, z, m, x_km, y_km

###Swarms obtained by after ETAS & seismicity rate & checking step:
df_swarms = pd.read_csv('v0_d03_n50_every1_clusters/new_df_final_swarms_of_v0_d03_n50_every1.csv')

##Rename swarms after sorting along t1:
df_swarms = df_swarms.sort_values("t1")
df_swarms.index = np.arange(1, len(df_swarms)+1)


###Create a column of swarms number:
df0["swarms_num"] = np.zeros(len(df0)) -1


for i in range(0, len(df_swarms)):
    select_swarms = df0.index[(df0.t >= df_swarms.t1.iloc[i])&(df0.t <= df_swarms.t2.iloc[i])
                       & (df0.x >= df_swarms.x1.iloc[i])&(df0.x <= df_swarms.x2.iloc[i])
                       & (df0.y >= df_swarms.y1.iloc[i])&(df0.y <= df_swarms.y2.iloc[i])
                       & (df0.z >= df_swarms.z1.iloc[i])&(df0.z <= df_swarms.z2.iloc[i])]
    
    df0.swarms_num.iloc[select_swarms] = df_swarms.index[i]   
    
    
df0['swarms_num'] = df0['swarms_num'].astype(int)

print(np.unique(df0["swarms_num"].values)[1:])

###Remove all rows with a magnitude equal to a nan value:
df0 = df0.dropna()

df0.index = np.arange(0, len(df0))

###Define AVN MS:
A, V, N, df_AVN = AVN(df0)

###Rescale on Norcia event:
xN_km = df0.x_km - N.x_km ##lat
yN_km = df0.y_km - N.y_km ##lon

df0["xN_km"] = xN_km
df0["yN_km"] = yN_km

###Redefine AVN MS:
A, V, N, df_AVN = AVN(df0)

df0_m5 = df0[df0.m >= 5]

df0

In [ ]:
from obspy.imaging.beachball import beach

##get moment tensor informations : https://www.globalcmt.org/CMTsearch.html 

mtA = [-2.400, 0.533, 1.860, 0.016, -0.660, -0.985]
mtV = [-2.190, 0.302, 1.880, 0.188, -1.030, -0.652]
mtN = [-1.470, 0.233, 1.230, 0.196, -0.543, -0.627]

In [ ]:
from matplotlib.patches import Rectangle

with open('v0_d03_n50_every1_clusters/mean_all_A_primx_and_mean_all_A_primy.npy', 'rb') as f:
    mean_all_A_primx = np.load(f)
    mean_all_A_primy = np.load(f)

In [ ]:
%%time 

x = np.arange(40,-50-2,-2)
print(x, len(x))

y = np.arange(-60, 50+2, 2)
print(y, len(y))

##2kmx2km

matrix = np.zeros((len(x), len(y)))
print(np.shape(matrix))

for i in range(0, len(x)-1):
    select_data = df0[(df0.xN_km >= np.min([x[i], x[i+1]]))&(df0.xN_km <np.max([x[i], x[i+1]]))]

    for j  in range(0, len(y)-1):
        select_data = df0[(df0.xN_km >= np.min([x[i], x[i+1]]))&(df0.xN_km <np.max([x[i], x[i+1]]))
                     & (df0.yN_km >= np.min([y[j], y[j+1]]))&(df0.yN_km <np.max([y[j], y[j+1]]))]

        L = len(select_data)
        
        if L < 1:
            L = np.nan

        matrix[i,j] = L
        

In [ ]:
###Get max value:
matrix2 = matrix
matrix2=matrix2[~np.isnan(matrix2)]
np.max(matrix2)

In [ ]:
plt.figure(figsize=(12,15))
plt.subplot(1,2,1)
im = plt.imshow(matrix, cmap = 'Reds', extent=[np.min(x),np.max(x),np.min(y),np.max(y)])
plt.clim(0, np.max(matrix2))
cb = plt.colorbar(im,orientation='horizontal',  pad=0.05).set_label(label='Total number of earthquakes', size=20)

plt.scatter(df0_m5.yN_km, df0_m5.xN_km, s=msz_stars, marker='*', c='gold', edgecolors='k', linewidths=1)
plt.scatter(A.yN_km, A.xN_km, s=msz_stars, marker='*', c='tab:blue', edgecolors='tab:red', linewidths=1)
plt.scatter(V.yN_km, V.xN_km, s=msz_stars, marker='*', c='tab:orange', edgecolors='tab:red', linewidths=1)
plt.scatter(N.yN_km, N.xN_km, s=msz_stars, marker='*', c='tab:green', edgecolors='tab:red', linewidths=1)
ax=plt.gca()
bcA = beach(mtA, xy=(A.yN_km, A.xN_km), width=msz_beach, linewidth=1, facecolor='tab:blue')
bcA.set_zorder(1)
ax.add_collection(bcA)
bcV = beach(mtV, xy=(V.yN_km, V.xN_km), width=msz_beach, linewidth=1, facecolor='tab:orange')
bcV.set_zorder(1)
ax.add_collection(bcV)
bcN = beach(mtN, xy=(N.yN_km, N.xN_km), width=msz_beach, linewidth=1, facecolor='tab:green')
bcN.set_zorder(1)
ax.add_collection(bcN)

plt.xlabel("W-E (km)", fontsize=15)
plt.ylabel("N-S (km)", fontsize=15)
plt.tick_params(axis="both", labelsize=15)
plt.gca().set_aspect('equal', 'box')
plt.xlim(-30, 40)
plt.ylim(-50,35)
plt.grid()

plt.subplot(1,2,2)
plt.scatter(df0.yN_km[df0.m >= m_min], df0.xN_km[df0.m >= m_min], s=np.exp(df0.m[df0.m >= m_min]), facecolors='none', edgecolors='k', linewidths=1, alpha=0.5)

##swarms clusters:
list_swarms = np.unique(df0.swarms_num)[1:]
for i in list_swarms:
    if dfM[dfM.I.values == i].empty == False:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.yN_km.mean(), I.xN_km.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors='k', linewidths=2)
    
    else:
        I = df0[df0.swarms_num == i]
        plt.scatter(I.yN_km.mean(), I.xN_km.mean(), s=msz, marker='s', facecolors=cmap[int(i)], edgecolors=cmap[int(i)], linewidths=1)
    
for k in range(0, len(mean_all_A_primx)):
    if k == 1:
        #add rectangle
        plt.gca().add_patch(Rectangle((mean_all_A_primx[k], mean_all_A_primy[k]),44,20,
                            angle=32,
                            edgecolor='m',
                            facecolor='none',
                            ls='--',
                            lw=1))
        
    elif k == 3:
        #add rectangle
        plt.gca().add_patch(Rectangle((mean_all_A_primx[k], mean_all_A_primy[k]),44,20,
                            angle=32,
                            edgecolor='m',
                            facecolor='none',
                            ls='--',
                            lw=1))
    ##44,10, or 44,20,
    
plt.scatter(df0_m5.yN_km, df0_m5.xN_km, s=msz_stars, marker='*', c='gold', edgecolors='k', linewidths=1)
plt.scatter(A.yN_km, A.xN_km, s=msz_stars, marker='*', c='tab:blue', edgecolors='tab:red', linewidths=1)
plt.scatter(V.yN_km, V.xN_km, s=msz_stars, marker='*', c='tab:orange', edgecolors='tab:red', linewidths=1)
plt.scatter(N.yN_km, N.xN_km, s=msz_stars, marker='*', c='tab:green', edgecolors='tab:red', linewidths=1)
ax=plt.gca()
bcA = beach(mtA, xy=(A.yN_km, A.xN_km), width=msz_beach, linewidth=1, facecolor='tab:blue')
bcA.set_zorder(1)
ax.add_collection(bcA)
bcV = beach(mtV, xy=(V.yN_km, V.xN_km), width=msz_beach, linewidth=1, facecolor='tab:orange')
bcV.set_zorder(1)
ax.add_collection(bcV)
bcN = beach(mtN, xy=(N.yN_km, N.xN_km), width=msz_beach, linewidth=1, facecolor='tab:green')
bcN.set_zorder(1)
ax.add_collection(bcN)
plt.xlabel("W-E (km)", fontsize=15)
plt.ylabel("N-S (km)", fontsize=15)
plt.tick_params(axis="both", labelsize=15)
plt.gca().set_aspect('equal', 'box')
plt.xlim(-30, 40)
plt.ylim(-50,35)
plt.grid(True)

plt.tight_layout()
plt.savefig("double_maps.png")
plt.show()